In [2]:
import numpy as np
import re
import tensorflow as tf
import time

In [3]:
lines = open('movie_lines.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')
conversations = open('movie_conversations.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')

In [4]:
id2line = {}
for line in lines:
    temp = line.split(' +++$+++ ')
    if len(temp) == 5:
        id2line[temp[0]] = temp[4]

In [5]:
conversations_ids =[]
for conversation in conversations[:-1]:
    temp1 = conversation.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    conversations_ids.append(temp1.split(','))

In [6]:
questions = []
answers = []
for conversation in conversations_ids:
    for i in range(len(conversation) - 1):
        questions.append(id2line[conversation[i]])
        answers.append(id2line[conversation[i+1]])

In [7]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm","i am", text)
    text = re.sub(r"he's","he is", text)
    text = re.sub(r"she's","she is", text)
    text = re.sub(r"that's","that is", text)
    text = re.sub(r"what's","what is", text)
    text = re.sub(r"where's","where is", text)
    text = re.sub(r"'ve","have", text)
    text = re.sub(r"'re","are", text)
    text = re.sub(r"won't","would not", text)
    text = re.sub(r"can't","cannot", text)
    text = re.sub(r"[-()\"#/@;:<>{}+=~|.?,]","",text)
    return text

In [8]:
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))

In [9]:
clean_answers = []
for answer in answers:
    clean_answers.append(clean_text(answer))

In [10]:
word2count = {}
for question in clean_questions:
    for word in question.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1 
for answer in clean_answers:
    for word in answer.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1

In [11]:
threshold = 20
questionwords2int = {}
word_number = 0
for word,count in word2count.items():
    if count >= threshold:
        questionwords2int[word] = word_number
        word_number += 1

answerwords2int = {}
word_number = 0
for word,count in word2count.items():
    if count >= threshold:
        answerwords2int[word] = word_number
        word_number += 1

In [12]:
tokens = ['<PAD>','<EOS>','<OUT>','<SOS>']
for token in tokens:
    questionwords2int[token] = len(questionwords2int) + 1
for token in tokens:
    answerwords2int[token] = len(answerwords2int) + 1
    

In [13]:
answerint2word = {w_i: w for w,w_i in answerwords2int.items()}

In [14]:
for i in range(len(clean_answers)):
    clean_answers[i] += ' <EOS>'

In [15]:
questions_to_int = []
for question in clean_questions:
    ints = []
    for word in question.split():
        if word not in questionwords2int:
            ints.append(questionwords2int['<OUT>'])
        else:
            ints.append(questionwords2int[word])
    questions_to_int.append(ints)
 
#my name is red 

answers_to_int = []
for answer in clean_answers:
    ints = []
    for word in answer.split():
        if word not in answerwords2int:
            ints.append(answerwords2int['<OUT>'])
        else:
            ints.append(answerwords2int[word])
    answers_to_int.append(ints)
                       

In [16]:
sorted_clean_questions = []
sorted_clean_answers = []
for length in range(1,26):
    for i in enumerate(questions_to_int):
        if len(i[1]) == length:
            sorted_clean_questions.append(questions_to_int[i[0]])
            sorted_clean_answers.append(answers_to_int[i[0]])
        